In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.metrics import mean_absolute_error

In [ ]:
# f' = 0.26 * (x**2 + y**2) - 0.48*x*y
# e = (f' - f) ** 2 = (0.26 * (x ** 2 + y ** 2) - 0.48 * x * y - f) ** 2
# de/dx = 2 * f' * (0.52*x - 0.48*y)
# de/dy = de/dx

ff = lambda x, y: 10 * x**2 + 0.05 * y**2
df_dx = lambda x, y: 20*x
df_dy = lambda x, y: 0.2*y

x_start = 5.0
y_start = -50

In [ ]:
fig = plt.figure(figsize=[20,12])
ax = fig.gca(projection='3d')
x = y = np.arange(-4.0, 4.0000001, 0.1)
y = np.arange(0, 2, 0.1)
X, Y = np.meshgrid(x, y)
zs = np.array([ff(x,y) for x,y in zip(np.ravel(X), np.ravel(Y))])
Z = zs.reshape(X.shape)
Gx, Gy = np.gradient(Z) # gradients with respect to x and y
G = (Gx**2.0+Gy**2.0)**.5  # gradient magnitude
N = G/G.max()  # normalize 0..1

ax.plot_surface(X, Y, Z, cmap=plt.cm.coolwarm, antialiased=False, shade=False, 
                rstride=5, cstride=1, linewidth=0)
#ax.set_aspect(0.3)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

# Stochastic Gradient Descent

In [ ]:
class SGD():
    def __init__(self, lr=0.001, x=None, y=None):
        self.lr = lr
        if x==None or y==None:
            self.x = np.random.rand()*10-5
            self.y = np.random.rand()*10-5
        else:
            self.x = x
            self.y = y
    def step(self, lr = None):
        if not lr:
            lr = self.lr
        # derivative
        f = ff(self.x, self.y)
        dx = df_dx(self.x, self.y)
        dy = df_dy(self.x, self.y)
        self.x = self.x - lr*dx
        self.y = self.y - lr*dy
        return [self.x, self.y, dx, dy]
        

In [ ]:
np.random.seed(655324)
opt = SGD(x=x_start, y=y_start)
errors=[]
xs,ys, dxs,dys= [x_start],[y_start],[],[]
sns.set_context("talk")
for epochs in range(300):
    x, y, dx, dy = opt.step(lr=0.03333)
    xs.append(x)
    ys.append(y)
    dxs.append(dx)
    dys.append(dy)
    errors.append(ff(x,y))
plt.figure(figsize=[18,6])
plt.plot(errors)
plt.title("Error evolution over time. Minimum error obtained in {0} iterations: {1}".format(len(errors), min(errors)))
plt.xlabel("time (iterations)")
plt.ylabel("error")
plt.show()
errors_sgd = errors

### Spatial evolution

In [ ]:
plt.figure(figsize=[18,6])
plt.subplot(131)
plt.plot(xs)
plt.title("X parameter evolution")
plt.xlabel("iterations")
plt.subplot(132)
plt.plot(ys)
plt.title("Y parameter evolution")
plt.xlabel("iterations")
plt.subplot(133)
plt.plot(xs, ys)
plt.title("x/y evolution")
plt.xlabel("x")
plt.show()

### Dynamic evolution

In [ ]:
vel_xs=np.diff(xs)
vel_ys=np.diff(ys)
plt.figure(figsize=[18,6])
plt.subplot(131)
plt.plot(np.abs(vel_xs[0:100]))
plt.title("X parameter velocity (momentum; $v_x$)")
plt.xlabel("iterations")
plt.subplot(132)
plt.plot(np.abs(vel_ys[0:100]))
plt.title("Y parameter velocity (momentum; $v_y$)")
plt.xlabel("iterations")
plt.subplot(133)
plt.plot(np.sqrt(np.array(vel_xs[0:100])**2 + np.array(vel_ys[0:100])**2))
plt.title("Absolute velocity ($\sqrt{v_x^2 + v_y^2}$)")
plt.xlabel("x")
plt.show()

# Stochastic Gradient Descent with Momentum

In [ ]:
class SGD_momentum():
    def __init__(self, lr=0.001, beta=0.9, x=None, y=None):
        self.lr = lr
        if x == None or y == None:
            self.x = np.random.rand()*10-5
            self.y = np.random.rand()*10-5
        else:
            self.x = x
            self.y = y
        self.beta = beta
        self.vx = 0
        self.vy = 0
        
    def step(self, lr = None, beta=None):
        if type(lr) == type(None):
            lr = self.lr
        if type(beta) == type(None):
            beta = self.beta
        f = ff(self.x, self.y)
        dx = df_dx(self.x, self.y)
        dy = df_dy(self.x, self.y)
        
        self.vx = beta * self.vx + lr * dx
        self.vy = beta * self.vy + lr * dy
        self.x += - self.vx
        self.y += - self.vy
            
        return [self.x, self.y, dx, dy, self.vx, self.vy]



In [ ]:
np.random.seed(655324)
xs,ys,vel_xs, vel_ys, dxs,dys= [x_start],[y_start],[],[],[],[]

opt = SGD_momentum(x=x_start, y=y_start)
errors=[]
for epochs in range(300):
    x, y, dx, dy, vel_x, vel_y= opt.step(lr=0.03333, beta=.65)
    vel_xs.append(vel_x)
    vel_ys.append(vel_y)
    xs.append(x)
    ys.append(y)
    dxs.append(dx)
    dys.append(dy)
    errors.append(ff(x,y))
plt.figure(figsize=[18,6])
plt.plot(errors)
plt.title("Error evolution over time. Minimum error obtained in {0} iterations: {1}".format(len(errors), min(errors)))
plt.xlabel("time (iterations)")
plt.ylabel("error")
plt.show()

errors_momentum = errors

### Spatial evolution 

In [ ]:
plt.figure(figsize=[18,6])
plt.subplot(131)
plt.plot(xs[0:100])
plt.title("X parameter evolution")
plt.xlabel("iterations")
plt.subplot(132)
plt.plot(ys[0:100])
plt.title("Y parameter evolution")
plt.xlabel("iterations")
plt.subplot(133)
plt.plot(xs[0:100], ys[0:100])
plt.title("x/y evolution")
plt.xlabel("x")
plt.show()

### Dynamic evolution 

In [ ]:
plt.figure(figsize=[18,6])
plt.subplot(131)
plt.plot(np.abs(vel_xs[0:100]))
plt.title("X parameter velocity ($v_x$)")
plt.xlabel("iterations")
plt.subplot(132)
plt.plot(np.abs(vel_ys[0:100]))
plt.title("Y parameter velocity ($v_y$)")
plt.xlabel("iterations")
plt.subplot(133)
plt.plot(np.sqrt(np.array(vel_xs[0:100])**2 + np.array(vel_ys[0:100])**2))
plt.title("Absolute velocity ($\sqrt{v_x^2 + v_y^2}$)")
plt.xlabel("x")
plt.show()

# Global comparison

In [ ]:
plt.figure(figsize=[18,6])
plt.plot(errors_sgd)
plt.plot(errors_momentum)
plt.title("Error comparison among the optimizers shown above")
plt.ylabel("error")
plt.xlabel("time (iterations)")
plt.legend(labels=["SGD", "SGD + Momentum"])
plt.show()